# Dependences

In [9]:
include("../../src/struct_data.jl")
include("../../src/neighbor.jl")

nearest_neighbors (generic function with 1 method)

# Input Data

## Model Parameter

In [2]:
@time Model = ModelSet(
    TimeModel(
        tₛᵢₘ  = 10000.0,
        dt    = 0.5,
        nₖₙₙ  = 50,
        nₛₐᵥₑ = 100
    ),
    InputModel(
        outer_ratio = 0.8,
        path_input = "../../data/init/Sphere"
    )
)
dump(Model)

  0.000001 seconds (3 allocations: 112 bytes)
ModelSet
  Time: TimeModel
    tₛᵢₘ: Float64 10000.0
    dt: Float64 0.5
    nₖₙₙ: Int64 50
    nₛₐᵥₑ: Int64 100
  Input: InputModel
    outer_ratio: Float64 0.8
    path_input: String "../../data/init/Sphere"


## Initial Aggregate

In [3]:
@time @start_agg FusionAGG = FusionAggregate(
    [
        AggType(
            "HEK", 
            InteractionPar(
                Cubic(1.0,2.0,3.0), 
                ContractilePar(0.01)
            ),
            Float64.(readdlm("../../data/init/Sphere/15.0.xyz")[3:end,2:end]) |> cu
        )
    ], 
    Model
)
show_aggregates(FusionAGG)

  7.917265 seconds (13.08 M allocations: 1.133 GiB, 3.34% gc time, 5.57% compilation time)
========================= Type =======================


1-element Vector{AggType}:
 AggType("HEK", InteractionPar(Cubic{Float64}(1.0, 2.0, 3.0), ContractilePar(0.01)), 15.27f0, Float32[-1.5 -4.62 -13.88; 0.5 -4.62 -13.88; … ; 0.5 4.62 13.88; 2.5 4.62 13.88], CuArray{Float32, 2, CUDA.Mem.DeviceBuffer})

======================   Index =======================
Index of List of Aggregates


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  1  1  1  1  1  1  1  1  1  1  1  …  1  1  1  1  1  1  1  1  1  1  1  1

Index of Number of Aggregates


...

Contractile


ContractilePar(Float32[0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01  …  0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01])

Radius


1-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 15.27

------------------ Neighbors Size -------------------
idx      = (5008, 5008)
idx_red  = (21, 5008)
idx_sum  = (1, 5008)
idx_cont = (50, 5008)
------------------- Forces Size ---------------------
dX       = (5008, 3)
F        = (5008, 3)


# Program

In [10]:
println("------------------------ SIZE ------------------------")
println("r_max         = $(FusionAGG.Type[1].Interaction.Force.rₘₐₓ)")
println("Size X        = $(size(FusionAGG.Position))")
println("Size idx      = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_real  = $(size(FusionAGG.Simulation.Neighbor.idx))")
println("Size idx_sum   = $(size(FusionAGG.Simulation.Neighbor.idx_sum))")
println("Size idx_red   = $(size(FusionAGG.Simulation.Neighbor.idx_red))")
@time nearest_neighbors(FusionAGG)
println("------------------------ IDX -------------------------")
println("idx")
display(FusionAGG.Simulation.Neighbor.idx)
println("idx_red")
display(FusionAGG.Simulation.Neighbor.idx_red)
println("idx_sum")
display(FusionAGG.Simulation.Neighbor.idx_sum)
println("idx_cont")
display(FusionAGG.Simulation.Neighbor.idx_cont)

------------------------ SIZE ------------------------
r_max         = 3.0
Size X        = (5008, 3)
Size idx      = (5008, 5008)
Size idx_real  = (5008, 5008)
Size idx_sum   = (1, 5008)
Size idx_red   = (21, 5008)
Threads = (32, 32) | Blocks  = (157, 157)
Threads = 256 | Blocks  = 20
Threads = (32, 32) | Blocks  = (157, 157)
 20.699895 seconds (31.15 M allocations: 1.769 GiB, 2.96% gc time, 17.07% compilation time)
------------------------ IDX -------------------------
idx


5008×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 1  1  0   0   1   1   0   0   0   0  …     0     0     0     0     0     0
 2  2  2   0   0   2   2   0   0   0        0     0     0     0     0     0
 0  3  3   0   0   0   3   3   0   0        0     0     0     0     0     0
 0  0  0   4   4   0   0   0   4   4        0     0     0     0     0     0
 5  0  0   5   5   5   0   0   0   5        0     0     0     0     0     0
 6  6  0   0   6   6   6   0   0   0  …     0     0     0     0     0     0
 0  7  7   0   0   7   7   7   0   0        0     0     0     0     0     0
 0  0  8   0   0   0   8   8   0   0        0     0     0     0     0     0
 0  0  0   9   0   0   0   0   9   9        0     0     0     0     0     0
 0  0  0  10  10   0   0   0  10  10        0     0     0     0     0     0
 0  0  0   0  11  11   0   0   0  11  …     0     0     0     0     0     0
 0  0  0   0   0  12  12   0   0   0        0     0     0     0     0     0
 0  0  0   0   0   0  13  13   0   0

idx_red


21×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
  1   1   2   4   1   1   2   3   4  …  4949  4950  4951  4957  4958  4959
  2   2   3   5   4   2   3   7   9     4957  4958  4959  4964  4965  4966
  5   3   7   9   5   5   6   8  10     4958  4959  4960  4965  4966  4967
  6   6   8  10   6   6   7  13  15     4964  4965  4966  4971  4972  4973
 35   7  37  40  10   7   8  14  48     4965  4966  4967  4972  4973  4974
 36  36  38  41  11  11  12  44  49  …  4966  4967  4968  4973  4974  4975
 37  37  39  42  41  12  13  45  50     4997  4998  4999  5002  5003  5004
 42  38  44  49  42  42  43  46  57     4998  4999  5000  5003  5004  5005
 43  43  45  50  43  43  44  53  58     5002  5003  5004  5006  5006  5007
 51  44  53  58  50  44  45  54  66     5003  5004  5005  5007  5007  5008
  0  52   0   0  51  51  52  62   0  …  5004  5005  5008     0  5008     0
  0   0   0   0  59  52  53   0   0     5006  5007     0     0     0     0
  0   0   0   0   0  60  61   0   0     5007  5008

idx_sum


1×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 10  11  10  10  12  13  13  11  10  …  11  10  12  13  13  11  10  11  10

idx_cont


50×5008 CuArray{Int64, 2, CUDA.Mem.DeviceBuffer}:
 43  43  38   4   6  12   7   7   9  …  5002  4999  4968  4957  4958  5007
  5   7  44  58   5   7   2   8  50     4957  5005  4959  5007  4958  4973
 35  44  45  41  42  12  52  13  57     5006  4950  4960  5002  5006  4959
  6   1   2  42  59  44  45  53  57     5007  5004  4968  5006  5003  5005
 37  43   8   4   4  60   2   3  50     4957  4998  4960  5002  5007  5007
  2   7  44   9   6   7   2  46  57  …  4966  4966  4960  4965  5007  4966
  2   6  38   4   1   1  43  54   4     4957  5007  4967  4964  4966  5008
  6  43   8  41   1  44   6  46   9     4964  4958  4966  4971  5006  4973
 36  38  39   9  43  51  52   7  66     5004  5005  4959  4964  4966  4966
 36  52  39  50  50  42  52  53  58     4964  5005  4959  4964  4973  4966
 35  38  39  41  10  60  13   7  66  …  4966  5004  4968  4965  4974  4973
 35  52  45  58  10  42   2  45  50     4949  4966  4959  4957  5006  4973
 51   2  53  41  11  51   8  53  66     4964  4965